#Código final

###Import libs

In [11]:
import pandas as pd
import awswrangler as ws
from ta import add_all_ta_features
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

###Leitura dos dados

In [ ]:
# ========== OBTER DADOS ==========

df_total = wr.athena.read_sql_table(
      table='tb_b3_portifolio',
      database='refined_b3',
      ctas_approach=False,
      workgroup=workgroup
  )

In [ ]:
df_total.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
0,2020-01-02 00:00:00-03:00,9.038210,9.094496,8.978963,9.094496,37774500,0.0,0.0,PETR4
1,2020-01-03 00:00:00-03:00,9.147817,9.254463,9.020435,9.020435,71595600,0.0,0.0,PETR4
2,2020-01-06 00:00:00-03:00,9.014511,9.165592,8.872317,9.127081,81844000,0.0,0.0,PETR4
3,2020-01-07 00:00:00-03:00,9.130042,9.147816,9.026359,9.091532,32822000,0.0,0.0,PETR4
4,2020-01-08 00:00:00-03:00,9.091531,9.115230,8.958224,9.035246,48215600,0.0,0.0,PETR4


###Pré-processamento

In [7]:
# ========== PRÉ-PROCESSAMENTO ==========

# Aplicar indicadores técnicos
df_total = add_all_ta_features(
    df_total, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True
)

# Ordenar e criar colunas de previsão para os próximos N dias
df_total.sort_values(['Ticker', 'Date'], inplace=True)
n_dias_futuro = 5  # Número de dias que queremos prever
for i in range(1, n_dias_futuro + 1):
    df_total[f'Close_+{i}d'] = df_total.groupby('Ticker')['Close'].shift(-i)

# Remover linhas com valores ausentes
df_total.dropna(inplace=True)

# Selecionar colunas de entrada e saída
features = df_total.select_dtypes(include=['float64', 'int64']).drop(
    columns=[f'Close_+{i}d' for i in range(1, n_dias_futuro + 1)]
).columns.tolist()

X = df_total[features]
y = df_total[[f'Close_+{i}d' for i in range(1, n_dias_futuro + 1)]]

###Treinamento

In [ ]:
# ========== TREINAMENTO ==========
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Regressor com multi-output
modelo = MultiOutputRegressor(XGBRegressor())
modelo.fit(X_train, y_train)

###Avaliação

In [13]:
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"\n===== MÉTRICAS DE AVALIAÇÃO =====")
print(f"MAE  (Erro Médio Absoluto): {mae:.2f}")
print(f"MSE  (Erro Quadrático Médio): {mse:.2f}")
print(f"RMSE (Raiz do Erro Quadrático Médio): {rmse:.2f}")
print(f"R²   (Coeficiente de Determinação): {r2:.4f}")


===== MÉTRICAS DE AVALIAÇÃO =====
MAE  (Erro Médio Absoluto): 3.85
MSE  (Erro Quadrático Médio): 37.38
RMSE (Raiz do Erro Quadrático Médio): 6.11
R²   (Coeficiente de Determinação): 0.9456


###Previsão

In [14]:
# ========== PREVISÃO PARA OS PRÓXIMOS DIAS ==========
ultimos = df_total.groupby('Ticker').tail(1)

previsoes_finais = []

for _, row in ultimos.iterrows():
    entrada = row[features].values.reshape(1, -1)
    previsao = modelo.predict(entrada)[0]

    print(f"\nTicker: {row['Ticker']}")
    previsao_dict = {'Ticker': row['Ticker']}
    for i in range(n_dias_futuro):
        dia = f'Dia_+{i+1}'
        valor = round(previsao[i], 2)
        previsao_dict[dia] = valor
        print(f"  {dia}: R$ {valor}")

    previsoes_finais.append(previsao_dict)


Ticker: ABEV3
  Dia_+1: R$ 12.170000076293945
  Dia_+2: R$ 12.130000114440918
  Dia_+3: R$ 12.020000457763672
  Dia_+4: R$ 12.239999771118164
  Dia_+5: R$ 12.220000267028809

Ticker: AZUL4
  Dia_+1: R$ 1.25
  Dia_+2: R$ 1.1200000047683716
  Dia_+3: R$ 1.2100000381469727
  Dia_+4: R$ 1.2999999523162842
  Dia_+5: R$ 1.0299999713897705

Ticker: BBDC4
  Dia_+1: R$ 17.549999237060547
  Dia_+2: R$ 17.479999542236328
  Dia_+3: R$ 17.40999984741211
  Dia_+4: R$ 17.299999237060547
  Dia_+5: R$ 17.0

Ticker: ITUB4
  Dia_+1: R$ 38.56999969482422
  Dia_+2: R$ 38.25
  Dia_+3: R$ 38.29999923706055
  Dia_+4: R$ 38.02000045776367
  Dia_+5: R$ 37.130001068115234

Ticker: MGLU3
  Dia_+1: R$ 9.569999694824219
  Dia_+2: R$ 9.260000228881836
  Dia_+3: R$ 9.460000038146973
  Dia_+4: R$ 9.1899995803833
  Dia_+5: R$ 8.979999542236328

Ticker: PETR4
  Dia_+1: R$ 31.6299991607666
  Dia_+2: R$ 31.059999465942383
  Dia_+3: R$ 31.059999465942383
  Dia_+4: R$ 31.3700008392334
  Dia_+5: R$ 30.93000030517578

Ticker

In [15]:
# Converter previsões em DataFrame
df_previsoes = pd.DataFrame(previsoes_finais)
print("\n===== TABELA DE PREVISÕES =====")
print(df_previsoes)


===== TABELA DE PREVISÕES =====
  Ticker     Dia_+1     Dia_+2     Dia_+3     Dia_+4     Dia_+5
0  ABEV3  12.170000  12.130000  12.020000  12.240000  12.220000
1  AZUL4   1.250000   1.120000   1.210000   1.300000   1.030000
2  BBDC4  17.549999  17.480000  17.410000  17.299999  17.000000
3  ITUB4  38.570000  38.250000  38.299999  38.020000  37.130001
4  MGLU3   9.570000   9.260000   9.460000   9.190000   8.980000
5  PETR4  31.629999  31.059999  31.059999  31.370001  30.930000
6  RAIZ4   1.090000   1.320000   1.220000   1.290000   1.590000
7  VALE3  67.510002  59.509998  56.799999  52.889999  50.389999


###Salvando dados na tabela

In [ ]:
wr.s3.to_parquet(
        df=df_previsoes,
        path='fiap-postech-bigdataarchiture/refined_b3/tb_b3_previsoes/',
        dataset=True,
        database='refined_b3',
        table='tb_b3_portifolio_previsoes',
        mode="overwrite",
        compression="snappy"
    )